# Import packets

In [11]:
import pandas as pd
from datetime import datetime
import time 
import requests

from functions import *

# Parsing data

In [12]:
date_today = datetime.now().strftime("%Y-%m-%d")
print(date_today)

data_air_quality = [get_air_quality_data('norway/norway/oslo/hjortnes')]
print(data_air_quality)

data_weather = [get_weather_data('oslo',date_today)]

2023-01-14
norway/norway/oslo/hjortnes
dict_keys(['o3', 'pm10', 'pm25'])
[['2023-01-14', 25.0, 10.0, 22.2]]


# Data preparation

In [13]:
df_air_quality = get_air_quality_df(data_air_quality)

print(df_air_quality.columns)

df_weather = get_weather_df(data_weather)

df_weather.head()

Index(['date', 'pm25', 'pm10', 'no2'], dtype='object')


,date,tempmax,tempmin,temp,dew,humidity,precip,precipcover,snow,snowdepth,windgust,windspeed,winddir,cloudcover,visibility,uvindex,conditions
0,1673625600000,1.2,-3.7,-0.4,-1.5,92.7,11.3,45.83,10.6,11.1,48.6,16.9,15.3,80.5,7.0,0,"Snow, Partially cloudy"


# Connect to Hopsworks feature store

In [14]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

air_quality_fg = fs.get_or_create_feature_group(
    name = 'oslo_air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'oslo_weather_fg',
    version = 1
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5316
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
# modify downloaded data type to make it compatible with the existed feature group
df_weather['conditions'] = df_weather['conditions'].replace(['Rain','Clear','Snow','Partially cloudy','Overcast','Snow, Partially cloudy',
                                                             'Rain, Partially cloudy','Rain, Overcast','Snow, Overcast',
                                                             'Snow, Freezing Drizzle/Freezing Rain, Overcast','Snow, Rain',
                                                             'Snow, Rain, Freezing Drizzle/Freezing Rain, Ice, Overcast',
                                                             'Snow, Rain, Freezing Drizzle/Freezing Rain, Overcast','Snow, Rain, Ice, Overcast',
                                                             'Snow, Rain, Overcast','Snow, Rain, Partially cloudy'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

In [16]:
for index, row in df_air_quality.iterrows():
    aqi = max(row['pm25'],row['pm10'],row['no2'])
    df_air_quality.loc[index,'aqi'] = aqi

df_air_quality

,date,pm25,pm10,no2,aqi
0,1673625600000,25.0,10.0,22.2,25.0


# Upload new data to the feature group

In [17]:
air_quality_fg.insert(df_air_quality)

weather_fg.insert(df_weather)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5316/jobs/named/oslo_air_quality_fg_1_offline_fg_backfill/executions


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5316/jobs/named/oslo_weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1cf16774250>, None)